# 6章 デプロイ

## 6.1 継続的インテグレーションとは

* CIの主な目的はすべてを互いに常に同期させること
  * 実際には新たにチェックインされたコードが既存コードと適切に統合（インテグレーション）されるようにすること


* 理想的には一度だけのビルドで、そのバージョンの他のコードのすべてのデプロイで再利用したい


* CIの利点
  * コードの品質に関して高速なフィードバック
  * バイナリ作成物の作成を自動化
  * 必要に応じて成果物を再作成可能
  * 成果物からコードへのある程度のトレーサビリティ

## 6.1.1 実際にCIを行っているか

* CIを行っていると言っているにもかかわらず実際には全く行っていない多くのチーム
  * CIツールの利用とCIのプラクティスを混同している


* CIとは何かを本当に理解しているかを調べるための3つの質問 (Jez Humble)
  * 1日に一度はメインラインにチェックインしていますか。
    * メインブランチに頻繁にマージすること
  * 変更を検証するテストスイートがありますか。
  * ビルドが壊れたときに、それを修正するのがチームの最優先事項でしょうか。
    * 壊れたビルドの修正とは関係ない変更を増やしていくと、ビルドを修正するためにかかる時間が増えていく

## 6.2 継続的インテグレーションとマイクロサービスへのマッピング

* CIビルドを個々のマイクロサービスにどのようにマッピングするか
  * 残りのサービスとは独立して、1つのサービスに変更を行いそれをデプロイできるようにしたい


* 勧められないが最も簡単な方法はすべてをひとまとめにすること
  * 全コードを格納する巨大なリポジトリと1つのビルド
  * 同時リリースを行う場合にはうまく機能する
    * 同時リリースは一般的には絶対避けるべき
    * プロジェクトの初期段階など短期的には理にかなっていることもある
  * 欠点
    * 1つのサービスを変更すると他のサービスもすべて検証されてビルドされる
    * 関係のないビルドのために時間が大幅にかかる
    * どれをデプロイすべきか判断しにくい
    * ビルドを壊してしまったときに他のサービスの変更ができなくなる
    * ビルドの責任を誰が負うか
  * この方法のバリエーションは複数のCIビルドをソースツリーの一部に対応付ける方法
    * 複数サービスのソースコードを一度にチェックインする習慣がとてもつきやすくなり、サービスを結合する変更にも陥りやすくなる (?)


* 代替手段: マイクロサービスごとに1つのCIビルドを持つこと

## 6.3 ビルドパイプラインと継続的デリバリ

* ビルドには複数のステージがあると便利
  * 多数の高速でスコープの小さいテストと少数の低速でスコープの大きいテスト
  * 高速のテストで失敗したら低速のテストをやる意味はあまりない
  * ビルドパイプラインを作成する